In [ ]:
#@title Data retrieval
import os, requests

fname = []
for j in range(3):
  fname.append('steinmetz_part%d.npz'%j)
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

for j in range(len(url)):
  if not os.path.isfile(fname[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)


# New Section

In [ ]:
#@title Data loading
import numpy as np

alldat = np.array([])
for j in range(len(fname)):
  alldat = np.hstack((alldat, np.load('steinmetz_part%d.npz'%j, allow_pickle=True)['dat']))

# select just one of the recordings here. 11 is nice because it has some neurons in vis ctx. 
dat = alldat[11]
print(dat.keys())

dict_keys(['spks', 'wheel', 'pupil', 'response', 'response_time', 'bin_size', 'stim_onset', 'contrast_right', 'contrast_left', 'brain_area', 'feedback_time', 'feedback_type', 'gocue', 'mouse_name', 'date_exp', 'trough_to_peak', 'active_trials', 'contrast_left_passive', 'contrast_right_passive', 'spks_passive', 'pupil_passive', 'wheel_passive', 'prev_reward', 'ccf', 'ccf_axes', 'cellid_orig', 'reaction_time', 'face', 'face_passive', 'licks', 'licks_passive'])


## Define Class

In [ ]:
from typing import List, Dict, Any
import numpy as np
from numpy import ndarray
import pandas as pd

class Session:

  def __init__(self, dat: Dict[str, Any]):
    self.dat = dat
    self.are_correct: ndarray
  
  def check_correct_res(self):
    """Make a list of correct responses"""
    n_trial = self.dat['spks'].shape[1]
    self.are_correct = np.zeros(n_trial)
    for i in range(n_trial):
      this_trial = SingleTrial(self.dat, i)
      self.are_correct[i] = this_trial.is_correct
    return self

class SingleTrial:
  def __init__(self, dat: Dict[str, Any], trial_idx: int):
    self.dat = dat
    self.idx = trial_idx
    self.is_correct: bool
    self.correct_res: int 
    if self.dat['contrast_left'][self.idx] < self.dat['contrast_right'][self.idx]:
      correct_res = -1
    elif self.dat['contrast_left'][self.idx] > self.dat['contrast_right'][self.idx]:
      correct_res = 1
    elif self.dat['contrast_left'][self.idx] == self.dat['contrast_right'][self.idx]:
      correct_res = 0
    else:
      raise ValueError('wrong!')
    self.correct_res = correct_res
    self.is_correct = self.dat['response'][self.idx] == correct_res

## Execute

In [ ]:
if __name__=='__main__':
  n_session: int = alldat.shape[0]

  df = pd.DataFrame(columns=['session_id', 'mouse_name',
                            'total', 'correct', 'incorrect',
                            'right_total',
                            'right_correct', 'right_incorrect',
                            'left_total',
                            'left_correct', 'left_incorrect',
                            'nogo_total',
                            'nogo_correct', 'nogo_incorrect'])

  for i in range(n_session):
    new_row: Dict[str, int] = {}
    session = Session(alldat[i]).check_correct_res()

    new_row['session_id'] = i
    new_row['mouse_name'] = session.dat['mouse_name']

    new_row['total'] = int(session.are_correct.shape[0])
    new_row['correct'] = int(np.sum(session.are_correct))
    new_row['incorrect'] = new_row['total'] - new_row['correct']
    
    right = session.are_correct[session.dat['response'] == -1]
    new_row['right_total'] = int(right.shape[0])
    new_row['right_correct'] = int(np.sum(right))
    new_row['right_incorrect'] = new_row['right_total'] - new_row['right_correct']

    left = session.are_correct[session.dat['response'] == 1]
    new_row['left_total'] = int(left.shape[0])
    new_row['left_correct'] = int(np.sum(left))
    new_row['left_incorrect'] = new_row['left_total'] - new_row['left_correct']

    nogo = session.are_correct[session.dat['response'] == 0]
    new_row['nogo_total'] = int(nogo.shape[0])
    new_row['nogo_correct'] = int(np.sum(nogo))
    new_row['nogo_incorrect'] = new_row['nogo_total'] - new_row['nogo_correct']

    df = df.append(new_row, ignore_index=True)

In [ ]:
df

,session_id,mouse_name,total,correct,incorrect,right_total,right_correct,right_incorrect,left_total,left_correct,left_incorrect,nogo_total,nogo_correct,nogo_incorrect
0,0,Cori,214,139,75,69,52,17,71,51,20,74,36,38
1,1,Cori,251,151,100,75,46,29,82,57,25,94,48,46
2,2,Cori,228,140,88,94,57,37,57,35,22,77,48,29
3,3,Forssmann,249,167,82,72,55,17,51,44,7,126,68,58
4,4,Forssmann,254,157,97,97,62,35,53,39,14,104,56,48
5,5,Forssmann,290,205,85,113,77,36,85,64,21,92,64,28
6,6,Forssmann,252,162,90,116,63,53,74,50,24,62,49,13
7,7,Hench,250,158,92,58,49,9,99,74,25,93,35,58
8,8,Hench,372,254,118,92,72,20,146,120,26,134,62,72
9,9,Hench,447,273,174,101,66,35,191,127,64,155,80,75
